In [ ]:
import rospy
from sensor_msgs.msg import Image
from cv_bridge import CvBridge, CvBridgeError

bridge = CvBridge()

In [ ]:
rospy.init_node('turtlebot_camera')

In [ ]:
def callback_color(msg):
    global color_img
    global color_timestamp
    color_timestamp = msg.header.stamp
    color_img = bridge.imgmsg_to_cv2(msg, "rgb8")

In [ ]:
subscriber_color = rospy.Subscriber("/camera/rgb/image_rect_color", Image, callback_color)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.imshow(color_img);

In [ ]:
plt.imshow(color_img[280:380,300:400,:]);

In [ ]:
color_img[340,320,:]

In [ ]:
color_img[340,350,:]

In [ ]:
import numpy as np
import cv2

In [ ]:
lower_blue = (0,0,120)
upper_blue = (100,100,255)
lower_red = (200,0,0)
upper_red = (255,100,100)

In [ ]:
mask_blue = cv2.inRange(color_img, lower_blue, upper_blue)
mask_red = cv2.inRange(color_img, lower_red, upper_red)

In [ ]:
plt.subplot(131); plt.imshow(color_img);plt.axis('off');
plt.subplot(132); plt.imshow(mask_blue,cmap='gray'); plt.axis('off');
plt.subplot(133); plt.imshow(mask_red,cmap='gray'); plt.axis('off');

In [ ]:
kernel = np.ones((5,5),np.uint8)
erosion_blue = cv2.erode(mask_blue,kernel,iterations = 1)
dilation_blue = cv2.dilate(erosion_blue,kernel,iterations = 3)
erosion_red = cv2.erode(mask_red,kernel,iterations = 1)
dilation_red = cv2.dilate(erosion_red,kernel,iterations = 3)
mask = cv2.bitwise_or(dilation_blue, dilation_red)

In [ ]:
plt.subplot(131); plt.imshow(dilation_blue,cmap='gray'); plt.axis('off');
plt.subplot(132); plt.imshow(dilation_red,cmap='gray'); plt.axis('off');
plt.subplot(133); plt.imshow(mask,cmap='gray'); plt.axis('off');

In [ ]:
im, cnt, hrc = cv2.findContours(mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnt.sort(key=cv2.contourArea, reverse=True)

In [ ]:
img = color_img.copy()
cv2.drawContours(img, cnt, -1, (0,255,0), 3)
plt.imshow(img);

In [ ]:
x, y, w, h = cv2.boundingRect( cnt[0] )
cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,255), 2)
plt.imshow(img);